# Cursors
SQL works based on set e.g., `SELECT` statement returns a set of rows which is called a `result set`. However, sometimes, you may want to process a data set on a `row by row` basis. This is where `cursors` come into play.

## Lifecycle
![Cursor Lifecycle](images/sql-server-cursor.png)
1. `DECLARE` your `variables` that you need in the logic.
2. `DECLARE` a `CURSOR` with a specific name that you will use throughout the logic. This is immediately followed by opening the cursor.
3. `FETCH` a record from cursor to begin the data processing.
4. Start a `WHILE` loop to check the `@@FETCHSTATUS`
    5. Is the data process that is unique to each set of logic. This could be inserting, updating, deleting, etc. for each row of data that was fetched. This is the most important set of logic during this process that is performed on each row.
    6. `FETCH` the next record from cursor as you did in step 3 and then step 5 is repeated again by processing the selected data.
7. Once all of the data has been processed, then you `CLOSE` the cursor to remove the lock on the rows (if the cursor has an `FOR UPDATE` statement)
8. Finally, you need to `DEALLOCATE` the cursor to release all of the internal resources SQL Server is holding.

## Declaration
To declare a cursor, you specify its name after the `DECLARE` keyword with the `CURSOR` data type and provide a `SELECT` statement that defines the result set for the cursor.

```sql
DECLARE <cursor_name> [...] CURSOR FOR
<SELECT_statement> 
[FOR {READ ONLY | UPDATE[OF <column list>]}]
```

- `READ ONLY` : Prohibits data  changes in underlying tables through cursor
- `UPDATE` : Data changes are allowed, optionally specify columns that can be changed via the cursor


### Open
Next, open and populate the cursor by executing the `SELECT` statement:

```sql
OPEN cursor_name;
```



### Fetching data
Then, fetch a row from the cursor into one or more variables:


```sql
FETCH NEXT FROM cursor INTO variable_list;
```

SQL Server provides the `@@FETCHSTATUS` function that returns the status of the last cursor `FETCH` statement executed against the cursor.

| Status Code | Description |
| ----------- | ----------- |
|  0          |	The FETCH statement was successful. |
| -1          | The FETCH statement failed or the row was beyond the result set. |
| ...         | Other statuscodes can be found online. |



### Iterate through the rows
```sql
WHILE @@FETCH_STATUS = 0  
    BEGIN
        -- Do something usefull here
        FETCH NEXT FROM cursor_name;  
    END;
```

### Close and Deallocate the cursor
```sql
CLOSE cursor_name;
```

```sql
DEALLOCATE cursor_name;
```

# Case 
`PRINT` the first 10 products of the company, in the following format:
> Product:1101 - Active Outdoors Crochet Glove has a price of 15.09
>
> Notice that a `SELECT` is different.

In [ ]:
DECLARE 
 @name VARCHAR(100)
,@price DECIMAL(7,2)
,@productid INT

DECLARE product_cursor CURSOR FOR 
    SELECT TOP 10
     Product.ProductName
    ,Product.ProductID
    ,Product.Price
    FROM Product
    ORDER BY ProductId ASC

OPEN product_cursor;

-- Read the first -possible- record into the variables
FETCH NEXT FROM product_cursor INTO
 @name
,@productid
,@price

-- Check if the last fetch operator was successfull
WHILE @@FETCH_STATUS = 0
BEGIN
    -- Do stuff
    PRINT FORMATMESSAGE('Product:%i - %s has a price of %s',@productid, @name, CONVERT(VARCHAR(7),@price));
    -- Read the next record into the variables.
    FETCH NEXT FROM product_cursor INTO
     @name
    ,@productid
    ,@price
END

CLOSE product_cursor;
DEALLOCATE product_cursor;

## Example
Printing some order information

In [ ]:
DECLARE 
 @orderid INT
,@orderamount DECIMAL(18,2)
,@customername nvarchar(40);

DECLARE orders_cursor CURSOR FOR
    SELECT 
     OrderID
    ,OrderAmount
    ,customername 
    FROM orders o 
        JOIN customer c on o.CustomerID = c.customerid
    WHERE OrderAmount > 10000;	

OPEN orders_cursor

FETCH NEXT FROM orders_cursor INTO 
 @orderid
,@orderamount
,@customername

WHILE @@FETCH_STATUS = 0 
BEGIN 
    PRINT FORMATMESSAGE('Order: %s, Amount: %s, Custoner: %s'
                        ,STR(@orderid)
                        ,STR(@orderamount)
                        ,@customername)
    FETCH NEXT FROM orders_cursor INTO 
     @orderid
    ,@orderamount
    ,@customername
END 

CLOSE orders_cursor

DEALLOCATE orders_cursor


# Example
Printing Order and OrderDetails information for every order.

In [ ]:
-- Outer Cursor Variables
DECLARE 
 @orderid INT
,@orderamount DECIMAL(18,2)
,@customername nvarchar(40);


-- Inner Cursor Variables
DECLARE 
 @productid int
,@quantity int

DECLARE orders_cursor CURSOR FOR
    SELECT 
     OrderID
    ,OrderAmount
    ,customername 
    FROM orders o 
        JOIN customer c on o.CustomerID = c.customerid
    WHERE OrderAmount > 10000;	

OPEN orders_cursor

FETCH NEXT FROM orders_cursor INTO 
 @orderid
,@orderamount
,@customername

WHILE @@FETCH_STATUS = 0 
BEGIN 
    PRINT FORMATMESSAGE('Order: %s, Amount: %s, Custoner: %s'
                        ,STR(@orderid)
                        ,STR(@orderamount)
                        ,@customername)
        -- Inner Cursor for OrderDetails
        DECLARE details_cursor CURSOR FOR 
            SELECT 
             productid
            ,quantity
            FROM OrdersDetail 
            WHERE OrderID = @orderid;

        OPEN details_cursor

        FETCH NEXT FROM details_cursor INTO 
         @productid
        ,@quantity

        WHILE @@FETCH_STATUS = 0 
        BEGIN 
            PRINT '-------' + 'Product: ' + str(@productid) + ', Quantity:' + str(@quantity)
            FETCH NEXT FROM details_cursor INTO 
             @productid
            ,@quantity
        END

        CLOSE details_cursor
        DEALLOCATE details_cursor
        -- End inner cursor
    FETCH NEXT FROM orders_cursor INTO 
     @orderid
    ,@orderamount
    ,@customername
END 

CLOSE orders_cursor

DEALLOCATE orders_cursor


# Mutating inside a cursor


In [ ]:
 WHILE @@FETCH_STATUS = 0 
  BEGIN 
    PRINT '      ' + 'Product: ' + str(@productid) + ', Quantity:' + str(@quantity)
    IF @quantity < 5
  DELETE FROM ordersdetail WHERE CURRENT OF details_cursor
    FETCH NEXT FROM details_cursor INTO @productid, @quantity
  END
